# Análise Exploratória dos Dados (EDA)

Este notebook apresenta a análise exploratória das bases de dados do desafio de análise
de crédito, com foco na compreensão das variáveis, distribuição da variável alvo e
qualidade dos dados.

Os dados utilizados neste notebook foram previamente ingeridos e armazenados em
formato Parquet pelo notebook `00_data_ingestion.ipynb`.


In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

In [4]:
PROJECT_ROOT = Path.cwd().parent
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

DATA_PROCESSED_DIR

WindowsPath('c:/Users/ggrinspun/Github/desafio-analise-de-credito/data/processed')

In [5]:
df_train = pd.read_parquet(DATA_PROCESSED_DIR / "train.parquet")
df_test  = pd.read_parquet(DATA_PROCESSED_DIR / "test.parquet")
df_oot   = pd.read_parquet(DATA_PROCESSED_DIR / "oot.parquet")

print("Train:", df_train.shape)
print("Test :", df_test.shape)
print("OOT  :", df_oot.shape)

Train: (120750, 151)
Test : (51751, 151)
OOT  : (91965, 150)
